@Author : kkutysllb

@E-mail : libing1@sn.chinamobile.com，31468130@qq.com

@Date   : 2024-11-26 10:15

@Desc   : AleNet图片分类

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
import sys
import os
sys.path.append('../')
from kk_libraries.kk_functions import get_device, kk_animator_train_evaluate
from kk_libraries.kk_models import kk_init_weights_relu
from kk_libraries.kk_dataprocess import kk_load_data, kk_predict_images_labels
from kk_libraries.kk_constants import text_labels_cifar10, mean, std


In [ ]:
# 定义模型
class AlexNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(AlexNet, self).__init__()

In [8]:
# 定义数据预处理
def kk_data_transform():
    return {
        'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean, std)]),
        'valid': transforms.Compose([transforms.Resize(256), 
                                     transforms.CenterCrop(224), 
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean, std)])
    }

In [9]:
# 数据加载
data_path = os.path.join('../', "data/CIFAR10")
train_loader, valid_loader, test_loader = kk_load_data(data_path, ratio=0.05, batch_size=256, DataSets=CIFAR10, transform=kk_data_transform())

2.0%